<a href="https://colab.research.google.com/github/sankarvinayak/DL-assignment2/blob/main/DA6401_DL_assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem Statement

In Part A and Part B of this assignment you will build and experiment with CNN based image classifiers using a subset of the [iNaturalist dataset](https://storage.googleapis.com/wandb_datasets/nature_12K.zip).



### Download the data and extract to the current directory


In [ ]:
!wget https://storage.googleapis.com/wandb_datasets/nature_12K.zip
!unzip -q nature_12K.zip
!pip install lightning

--2025-04-05 08:52:12--  https://storage.googleapis.com/wandb_datasets/nature_12K.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.130.207, 64.233.170.207, 74.125.68.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.130.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3816687935 (3.6G) [application/zip]
Saving to: ‘nature_12K.zip’

nature_12K.zip      100%[===================>]   3.55G  22.4MB/s    in 2m 44s  

2025-04-05 08:54:56 (22.3 MB/s) - ‘nature_12K.zip’ saved [3816687935/3816687935]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 818.9/818.9 kB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os, random,torch,torchvision
import torch.nn as nn
import torch.functional as F
import torch.optim as optim
import pytorch_lightning as pl
from torchvision import transforms, datasets
from PIL import Image
from torch.utils.data import DataLoader, random_split
from torchvision import models
from pytorch_lightning.loggers import WandbLogger


# Part A: Training from scratch

## Question 1 (5 Marks)
Build a small CNN model consisting of $5$ convolution layers. Each convolution layer would be followed by an activation and a max-pooling layer.

After $5$ such conv-activation-maxpool blocks, you should have one dense layer followed by the output layer containing $10$ neurons ($1$ for each of the $10$ classes). The input layer should be compatible with the images in the [iNaturalist dataset](https://storage.googleapis.com/wandb_datasets/nature_12K.zip) dataset.

The code should be flexible such that the number of filters, size of filters, and activation function of the convolution layers and dense layers can be changed. You should also be able to change the number of neurons in the dense layer.

- What is the total number of computations done by your network? (assume $m$ filters in each layer of size $k\times k$  and $n$ neurons in the dense layer)
- What is the total number of parameters in your network? (assume $m$ filters in each layer of size $k\times k$ and $n$ neurons in the dense layer)


### Create dataloader from the dataset

In [ ]:
class iNaturalistDataModule(pl.LightningDataModule):
    def __init__(self, train_dir: str,test_dir: str, batch_size: int=128, num_workers: int = 2,train_transforms=transforms.ToTensor(), test_transforms=transforms.ToTensor(), train_val_split: float = 0.8,seed=41):
      super().__init__()

      self.save_hyperparameters()
      self.train_dir = train_dir
      self.test_dir = test_dir
      self.batch_size = batch_size
      self.num_workers = num_workers
      self.train_val_split = train_val_split

      self.train_transforms = train_transforms
      self.test_transforms=test_transforms
      self.seed=seed

    def setup(self, stage=None):
      torch.manual_seed(self.seed)
      torch.cuda.manual_seed(self.seed)
      train_dataset = datasets.ImageFolder(root=self.train_dir, transform=self.train_transforms)
      total_size = len(train_dataset)
      train_size = int(total_size * self.train_val_split)
      val_size = total_size - train_size
      self.train_dataset, self.val_dataset = random_split(train_dataset, [train_size, val_size])

      self.test_dataset = datasets.ImageFolder(root=self.test_dir, transform=self.test_transforms)

    def train_dataloader(self):
      return DataLoader(self.train_dataset,batch_size=self.batch_size,shuffle=True,num_workers=self.num_workers)

    def val_dataloader(self):

      return DataLoader(self.val_dataset,batch_size=self.batch_size,shuffle=False, num_workers=self.num_workers)

    def test_dataloader(self):
      return DataLoader(self.test_dataset,batch_size=self.batch_size,shuffle=False,num_workers=self.num_workers)




In [ ]:
transform=transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor()])
naturalist_DM=iNaturalistDataModule(train_dir='inaturalist_12K/train',test_dir='inaturalist_12K/val',train_transforms=transform,test_transforms=transform)
naturalist_DM.setup()

In [ ]:
train_loader=naturalist_DM.train_dataloader()
val_loader=naturalist_DM.val_dataloader()
test_loader=naturalist_DM.test_dataloader()

In [ ]:

img = Image.open("inaturalist_12K/train/Amphibia/02f95591e712f05cae136a91a4d73ea5.jpg")
width, height = img.size
print(f"Width: {width}, Height: {height}")


Width: 800, Height: 462


In [ ]:
import torch
mean = torch.zeros(3)
std = torch.zeros(3)
total_samples = 0
for images, _ in train_loader:
    batch_samples = images.size(0)
    images = images.view(batch_samples, 3, -1)

    mean += images.mean(dim=2).sum(dim=0)
    std += images.std(dim=2).sum(dim=0)
    total_samples += batch_samples

mean /= total_samples
std /= total_samples

print(f"Computed Mean: {mean.tolist()}")
print(f"Computed Std: {std.tolist()}")


Computed Mean: [0.47089460492134094, 0.45923930406570435, 0.3884953558444977]
Computed Std: [0.19317267835140228, 0.18763333559036255, 0.1841067522764206]


In [ ]:
train_loader = naturalist_DM.train_dataloader()
shape=None
for images, labels in train_loader:
    print("Batch image shape:", images.shape)
    shape=images.shape
    break
shape

Batch image shape: torch.Size([32, 3, 224, 224])


torch.Size([32, 3, 224, 224])

In [ ]:


class iNaturalistModel(pl.LightningModule):
    def __init__(self, in_channels=3,input_size=(224, 224),dense_size=512,  num_filters_layer: list = [16, 32, 64, 128, 256],filter_size: list = [3, 3, 3, 3, 3], stride=1,  padding=1, num_classes=10,activation=nn.ReLU,dropout_rate=0,optimizer=torch.optim.AdamW,lr=1e-3,batch_norm=False ):
        super().__init__()
        self.save_hyperparameters()
        self.optimizer=optimizer
        self.lr=lr
        layers = []
        size = input_size
        activation_name=activation.__name__.lower()
        for num_channel, k in zip(num_filters_layer, filter_size):
            conv=nn.Conv2d(in_channels=in_channels,out_channels=num_channel,kernel_size=k,stride=stride,padding=padding)
            # if activation_name in ["relu", "mish"]:
            #     init.kaiming_normal_(conv.weight, nonlinearity='relu')
            # elif activation_name in ["silu", "gelu"]:
            #     init.lecun_normal_(conv.weight)
            # else:
            #   init.xavier_normal_(conv.weight)
            # if conv.bias is not None:
            #     init.zeros_(conv.bias)
            layers.append(conv)

            if batch_norm:
              layers.append(nn.BatchNorm2d(num_features=num_channel))
            layers.append(activation())
            layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
            in_channels = num_channel
            conv_h = (size[0] + 2 * padding - k) // stride + 1
            conv_w = (size[1] + 2 * padding - k) // stride + 1
            pool_h = conv_h // 2
            pool_w = conv_w // 2
            size = (pool_h, pool_w)
        layers.append(nn.Flatten())
        if dropout_rate > 0:
            layers.append(nn.Dropout(dropout_rate))
        flattened_size = size[0] * size[1] * num_filters_layer[-1]
        fc1=nn.Linear(in_features=flattened_size, out_features=dense_size)
        # if activation_name=="relu":
        #   init.kaiming_normal_(fc1.weight, nonlinearity='relu')
        # else:
        #   init.xavier_normal_(fc1.weight)
        # if fc1.bias is not None:
        #     init.zeros_(fc1.bias)
        layers.append(fc1)
        layers.append(activation())
        if dropout_rate > 0:
            layers.append(nn.Dropout(dropout_rate))
        fc2=nn.Linear(in_features=dense_size, out_features=num_classes)
        # init.xavier_normal_(fc2.weight)
        # if fc2.bias is not None:
        #     init.zeros_(fc2.bias)
        layers.append(fc2)
        self.model = nn.Sequential(*layers)
    def forward(self, x):
        return self.model(x)
    def training_step(self, batch, batch_idx):
        images, labels = batch
        logits = self.forward(images)
        loss = nn.functional.cross_entropy(logits, labels)
        preds = torch.argmax(logits, dim=1)
        acc = (preds == labels).float().mean()
        self.log("train_acc", acc, prog_bar=True)
        self.log("train_loss", loss, prog_bar=True)
        return loss
    def validation_step(self, batch, batch_idx):
        images, labels = batch
        logits = self.forward(images)
        loss = nn.functional.cross_entropy(logits, labels)
        preds = torch.argmax(logits, dim=1)
        acc = (preds == labels).float().mean()
        self.log("validation_acc", acc, prog_bar=True)
        self.log("validation_loss", loss, prog_bar=True)
        return {"validation_loss":loss,"validation_acc":acc}
    def test_step(self, batch, batch_idx):
        images, labels = batch
        logits = self.forward(images)
        loss = nn.functional.cross_entropy(logits, labels)
        preds = torch.argmax(logits, dim=1)
        acc = (preds == labels).float().mean()
        self.log("test_acc", acc, prog_bar=True)
        self.log("test_loss", loss, prog_bar=True)
        return {"test_loss":loss,"test_acc":acc}
    def configure_optimizers(self):
        return self.optimizer(self.model.parameters(), lr=self.lr,weight_decay=1e-5)



## Question 2 (15 Marks)
You will now train your model using the [iNaturalist dataset](https://storage.googleapis.com/wandb_datasets/nature_12K.zip). The zip file contains a train and a test folder. Set aside $20\%$ of the training data, as validation data, for hyperparameter tuning. Make sure each class is equally represented in the validation data. **Do not use the test data for hyperparameter tuning.**

Using the sweep feature in wandb find the best hyperparameter configuration. Here are some suggestions but you are free to decide which hyperparameters you want to explore

- number of filters in each layer : $32$, $64$, ...
- activation function for the conv layers: ReLU, GELU, SiLU, Mish, ...
- filter organisation: same number of filters in all layers, doubling in each subsequent layer, halving in each subsequent layer, etc
- data augmentation: Yes, No
- batch normalisation: Yes, No
- dropout: $0.2$, $0.3$ (BTW, where will you add dropout? You should read up a bit on this)

Based on your sweep please paste the following plots which are automatically generated by wandb:
- accuracy v/s created plot (I would like to see the number of experiments you ran to get the best configuration).
- parallel co-ordinates plot
- correlation summary table (to see the correlation of each hyperparameter with the loss/accuracy)

Also, write down the hyperparameters and their values that you sweeped over. Smart strategies to reduce the number of runs while still achieving a high accuracy would be appreciated. Write down any unique strategy that you tried.


overfitting

In [ ]:
model=iNaturalistModel()
trainer=pl.Trainer(max_epochs=30)
trainer.fit(model,train_dataloaders=train_loader,val_dataloaders=val_loader)

INFO:pytorch_lightning.utilities.rank_zero:You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type       | Params | Mode 
---------------------------------------------
0 | model | Sequential | 6.8 M  | train
---------------------------------------------
6.8 M     Trainable params
0         Non-trainable params
6.8 M     Total params
27.283    Total estimated model params size (MB)
20        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:
Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined

too slow to improve

In [ ]:
model=iNaturalistModel(num_filters_layer = [64,128,256,512,1024],filter_size = [7, 5, 5, 3, 3], stride=1,  padding=1, num_classes=10,activation=torch.nn.GELU,dropout_rate=0.5,optimizer=torch.optim.AdamW,lr=1e-3,batch_norm=True )
trainer=pl.Trainer(max_epochs=30)
trainer.fit(model,train_dataloaders=train_loader,val_dataloaders=val_loader)

INFO:pytorch_lightning.utilities.rank_zero:You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type       | Params | Mode 
---------------------------------------------
0 | model | Sequential | 25.8 M | train
---------------------------------------------
25.8 M    Trainable params
0         Non-trainable params
25.8 M    Total params
103.270   Total estimated model params size (MB)
27        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:
Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined

In [ ]:
model=iNaturalistModel(dense_size=4096,num_filters_layer = [32,64,128,256,512],filter_size = [5, 5, 3, 3, 3], stride=1,  padding=1, num_classes=10,activation=torch.nn.ReLU,dropout_rate=0.5,optimizer=torch.optim.AdamW,lr=1e-2,batch_norm=True )
trainer=pl.Trainer(max_epochs=30)
naturalist_DM=iNaturalistDataModule(train_dir='inaturalist_12K/train',test_dir='inaturalist_12K/val',train_transforms=transform,test_transforms=transform,batch_size=64)
naturalist_DM.setup()
trainer.fit(model,datamodule=naturalist_DM)

In [ ]:
import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),  # Random crop for scale variation
    transforms.RandomHorizontalFlip(p=0.5),  # Flip for left-right symmetry
    transforms.RandomApply([transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1)], p=0.3),  # Color jitter for lighting variation
    transforms.RandomRotation(degrees=15),  # Small rotations to generalize better
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),  # Small translations
    transforms.RandomPerspective(distortion_scale=0.2, p=0.3),  # Perspective distortion

    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Standard ImageNet normalization
])
naturalist_DM=iNaturalistDataModule(train_dir='/content/inaturalist_12K/train',test_dir='/content/inaturalist_12K/val',train_transforms=transform,test_transforms=transform)
naturalist_DM.setup()

model=iNaturalistModel(dense_size=4096,num_filters_layer = [32,64,128,256,512],filter_size = [3, 3, 3, 3, 3], stride=1,  padding=1, num_classes=10,activation=torch.nn.ReLU,dropout_rate=0.5,optimizer=torch.optim.Adam,lr=0.1,batch_norm=True )
trainer=pl.Trainer(max_epochs=30)
trainer.fit(model,datamodule=naturalist_DM)



INFO:pytorch_lightning.utilities.rank_zero:You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type       | Params | Mode 
---------------------------------------------
0 | model | Sequential | 104 M  | train
---------------------------------------------
104 M     Trainable params
0         Non-trainable params
104 M     Total params
417.504   Total estimated model params size (MB)
27        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:
Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined

In [ ]:
import torch.nn.init as init


class iNaturalistModel(pl.LightningModule):
    def __init__(self, in_channels=3,input_size=(224, 224),dense_size=512,  num_filters_layer: list = [16, 32, 64, 128, 256],filter_size: list = [3, 3, 3, 3, 3], stride=1,  padding=1, num_classes=10,activation=nn.ReLU,dropout_rate=0,optimizer=torch.optim.AdamW,lr=1e-3,batch_norm=False ):
        super().__init__()
        self.save_hyperparameters()
        self.optimizer=optimizer
        self.lr=lr
        layers = []
        size = input_size
        activation_name=activation.__name__.lower()
        for num_channel, k in zip(num_filters_layer, filter_size):
            conv=nn.Conv2d(in_channels=in_channels,out_channels=num_channel,kernel_size=k,stride=stride,padding=padding)
            if activation_name in ["relu", "mish"]:
                init.kaiming_normal_(conv.weight, nonlinearity='relu')
            elif activation_name in ["silu", "gelu"]:
                init.kaiming_normal_(conv.weight, nonlinearity="linear")
            else:
              init.xavier_normal_(conv.weight)
            if conv.bias is not None:
                init.zeros_(conv.bias)
            layers.append(conv)

            if batch_norm:
              layers.append(nn.BatchNorm2d(num_features=num_channel))
            layers.append(activation())
            layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
            in_channels = num_channel
            conv_h = (size[0] + 2 * padding - k) // stride + 1
            conv_w = (size[1] + 2 * padding - k) // stride + 1
            pool_h = conv_h // 2
            pool_w = conv_w // 2
            size = (pool_h, pool_w)
        layers.append(nn.Flatten())
        if dropout_rate > 0:
            layers.append(nn.Dropout(dropout_rate))
        flattened_size = size[0] * size[1] * num_filters_layer[-1]
        fc1=nn.Linear(in_features=flattened_size, out_features=dense_size)
        if activation_name=="relu":
          init.kaiming_normal_(fc1.weight, nonlinearity='relu')
        else:
          init.xavier_normal_(fc1.weight)
        if fc1.bias is not None:
            init.zeros_(fc1.bias)
        layers.append(fc1)
        layers.append(activation())
        if dropout_rate > 0:
            layers.append(nn.Dropout(dropout_rate))
        fc2=nn.Linear(in_features=dense_size, out_features=num_classes)
        init.xavier_normal_(fc2.weight)
        if fc2.bias is not None:
            init.zeros_(fc2.bias)
        layers.append(fc2)
        self.model = nn.Sequential(*layers)
    def forward(self, x):
        return self.model(x)
    def training_step(self, batch, batch_idx):
        images, labels = batch
        logits = self.forward(images)
        loss = nn.functional.cross_entropy(logits, labels)
        preds = torch.argmax(logits, dim=1)
        acc = (preds == labels).float().mean()
        self.log("train_acc", acc, prog_bar=True)
        self.log("train_loss", loss, prog_bar=True)
        return loss
    def validation_step(self, batch, batch_idx):
        images, labels = batch
        logits = self.forward(images)
        loss = nn.functional.cross_entropy(logits, labels)
        preds = torch.argmax(logits, dim=1)
        acc = (preds == labels).float().mean()
        self.log("validation_acc", acc, prog_bar=True)
        self.log("validation_loss", loss, prog_bar=True)
        return {"validation_loss":loss,"validation_acc":acc}
    def test_step(self, batch, batch_idx):
        images, labels = batch
        logits = self.forward(images)
        loss = nn.functional.cross_entropy(logits, labels)
        preds = torch.argmax(logits, dim=1)
        acc = (preds == labels).float().mean()
        self.log("test_acc", acc, prog_bar=True)
        self.log("test_loss", loss, prog_bar=True)
        return {"test_loss":loss,"test_acc":acc}
    def configure_optimizers(self):
        return self.optimizer(self.model.parameters(), lr=self.lr,weight_decay=1e-5)


In [ ]:
trainer.test(model,test_loader)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.31349998712539673    │
│         test_loss         │    1.9300973415374756     │
└───────────────────────────┴───────────────────────────┘

[{'test_acc': 0.31349998712539673, 'test_loss': 1.9300973415374756}]

In [ ]:
import wandb
sweep_config = {
    "method": "bayes",
    "metric": {
        "name": "val_acc",
        "goal": "maximize"
    },
    "parameters": {
        "num_filter": {
            "values":[16,32,64]
        },
        "activation_fun": {
            "values":["ReLU", "GELU", "SiLU", "Mish"]
        },
        "filter_org": {
            "values":["same","double"]
        },
        "data_augmentation": {
            "values":["No"]
        },
        "augmentation_technique": {
            "values": ["None"]
        },
        "batch_norm": {
            "values":["No","Yes"]
        },
        "dropout": {
            "values":[0,0.2,0.3,0.5]
        },

        "filter_size": {
            "values":[3,5,7]
        },
        "epoch": {
            "values":[5,10,20,30,40,50]
        },
        "batch_size": {
            "values":[32,64,128,256]
        },"dene_size": {
            "values":[512,1024,2046,4096]
        }
    }
}
sweep_id = wandb.sweep(sweep_config, project="DL-Addignemt2_A")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: tt826w5h
Sweep URL: https://wandb.ai/cs24m041-iit-madras/DL-Addignemt2_A/sweeps/tt826w5h


In [ ]:
import wandb
sweep_config = {
    "method": "bayes",
    "metric": {
        "name": "val_acc",
        "goal": "maximize"
    },
    "parameters": {
        "num_filter": {
            "values":[16,32,64]
        },
        "activation_fun": {
            "values":["ReLU", "GELU", "SiLU", "Mish"]
        },
        "filter_org": {
            "values":["same","double"]
        },
        "data_augmentation": {
            "values":["Yes"]
        },
        "augmentation_technique": {
            "values": ["RandomFlip", "RandomCrop", "ColorJitter"]
        },
        "batch_norm": {
            "values":["No","Yes"]
        },
        "filter_size": {
            "values":[3,5,7]
        },
        "dropout": {
            "values":[0,0.2,0.3,0.5]
        },
        "epoch": {
            "values":[5,10,20,30,40,50,60]
        },
        "batch_size": {
            "values":[32,64,128,256]
        },"dene_size": {
            "values":[512,1024,2046,4096]
        }
    }
}
sweep_id = wandb.sweep(sweep_config, project="DL-Addignemt2_A")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: 62k06x6w
Sweep URL: https://wandb.ai/cs24m041-iit-madras/DL-Addignemt2_A/sweeps/62k06x6w


In [ ]:


from pytorch_lightning.loggers import WandbLogger
def train_model():
    wandb.init()
    config = wandb.config
    run_name = f"Augment{config.data_augmentation}activation_fun{config.activation_fun}_dropout_{config.dropout}"
    wandb.run.name = run_name

    if config.data_augmentation == "Yes":
      if config.augmentation_technique == "RandomFlip":
        transform=transforms.Compose([
            transforms.Resize((224,224)),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.ToTensor(),transforms.Normalize(mean=[0.4709, 0.4592, 0.3885], std=[0.1932, 0.1876, 0.1841])
])
      elif config.augmentation_technique == "RandomCrop":
        transform=transforms.Compose([
            transforms.Resize((224,224)),
            transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
            transforms.ToTensor(),transforms.Normalize(mean=[0.4709, 0.4592, 0.3885], std=[0.1932, 0.1876, 0.1841])
])
      elif config.augmentation_technique == "ColorJitter":
        transform=transforms.Compose([
            transforms.Resize((224,224)),
            transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
            transforms.ToTensor(),transforms.Normalize(mean=[0.4709, 0.4592, 0.3885], std=[0.1932, 0.1876, 0.1841])
])
    else:
      transform=transforms.Compose([
            transforms.Resize((224,224)),
            transforms.ToTensor(),transforms.Normalize(mean=[0.4709, 0.4592, 0.3885], std=[0.1932, 0.1876, 0.1841])
])

    num_filters=config.num_filter
    num_filters_layer=[num_filters,num_filters*2,num_filters*4,num_filters*8,num_filters*16]if config.filter_org=="double" else [num_filters]*5
    if config.activation_fun=="ReLU":
      activation=torch.nn.ReLU
    elif config.activation_fun=="GELU":
      activation=torch.nn.GELU
    elif config.activation_fun=="SiLU":
      activation=torch.nn.SiLU
    elif config.activation_fun=="Mish":
      activation=nn.Mish
    filter_size=config.filter_size
    filter_sizes=[filter_size]*5
    if config.batch_norm=="Yes":
      model = iNaturalistModel(num_filters_layer=num_filters_layer,activation=activation,dropout_rate=config.dropout,batch_norm=True,filter_size=filter_sizes,dense_size=config.dene_size)
    else:
      model = iNaturalistModel(num_filters_layer=num_filters_layer,activation=activation,dropout_rate=config.dropout,filter_size=filter_sizes,dense_size=config.dene_size)
    print(model)



    wandb_logger = WandbLogger(project="DL-Addignemt2_A",name=run_name)
    trainer = pl.Trainer(logger=wandb_logger, max_epochs=config.epoch)

    test_tranform=transforms.Compose([
            transforms.Resize((224,224)),
            transforms.ToTensor()])
    naturalist_DM=iNaturalistDataModule(train_dir='/content/inaturalist_12K/train',test_dir='/content/inaturalist_12K/val',batch_size=config.batch_size,train_transforms=transform,test_transforms=test_tranform)
    trainer.fit(model, naturalist_DM)


In [ ]:
wandb.agent(sweep_id, function=train_model, count=10)


wandb: Agent Starting Run: 9lk0zl7f with config:
wandb: 	activation_fun: SiLU
wandb: 	augmentation_technique: RandomFlip
wandb: 	batch_norm: Yes
wandb: 	batch_size: 64
wandb: 	data_augmentation: Yes
wandb: 	dropout: 0.3
wandb: 	epoch: 60
wandb: 	filter_org: double
wandb: 	filter_size: 5
wandb: 	num_filter: 32
wandb: Currently logged in as: cs24m041 (cs24m041-iit-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


INFO:pytorch_lightning.utilities.rank_zero:You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


iNaturalistModel(
  (model): Sequential(
    (0): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): SiLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): SiLU()
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): SiLU()
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): Conv2d(128, 256, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
    (13): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=T

/usr/local/lib/python3.11/dist-packages/pytorch_lightning/loggers/wandb.py:397: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type       | Params | Mode 
---------------------------------------------
0 | model | Sequential | 10.9 M | train
---------------------------------------------
10.9 M    Trainable params
0         Non-trainable params
10.9 M    Total params
43.666    Total estimated model params size (MB)
26        Modules in train mode
0         Modules in eval mode
wandb: WARNING Config item 'filter_size' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'batch_norm' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'batch_size' was locked by 'sweep' (ignored upd

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

wandb: Ctrl + C detected. Stopping sweep.


In [ ]:
import wandb
wandb.agent("cs24m041-iit-madras/DL-Addignemt2_A/62k06x6w", function=train_model, count=1)


wandb: Agent Starting Run: powzezi9 with config:
wandb: 	activation_fun: GELU
wandb: 	augmentation_technique: ColorJitter
wandb: 	batch_norm: Yes
wandb: 	batch_size: 128
wandb: 	data_augmentation: Yes
wandb: 	dropout: 0.2
wandb: 	epoch: 30
wandb: 	filter_org: same
wandb: 	filter_size: 3
wandb: 	num_filter: 32


INFO:pytorch_lightning.utilities.rank_zero:You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


iNaturalistModel(
  (model): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): GELU(approximate='none')
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): GELU(approximate='none')
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): GELU(approximate='none')
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): BatchNorm2d(32, eps=1e-05,

/usr/local/lib/python3.11/dist-packages/pytorch_lightning/loggers/wandb.py:397: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type       | Params | Mode 
---------------------------------------------
0 | model | Sequential | 846 K  | train
---------------------------------------------
846 K     Trainable params
0         Non-trainable params
846 K     Total params
3.387     Total estimated model params size (MB)
26        Modules in train mode
0         Modules in eval mode
wandb: WARNING Config item 'filter_size' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'batch_norm' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'batch_size' was locked by 'sweep' (ignored upd

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [ ]:
import wandb
wandb.agent("cs24m041-iit-madras/DL-Addignemt2_A/wuoziom7", function=train_model, count=10)


wandb: Agent Starting Run: 8n4mv7br with config:
wandb: 	activation_fun: Mish
wandb: 	augmentation_technique: ColorJitter
wandb: 	batch_norm: No
wandb: 	data_augmentation: Yes
wandb: 	dropout: 0.2
wandb: 	filter_org: same
wandb: 	num_filter: 16


INFO:pytorch_lightning.utilities.rank_zero:You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type       | Params | Mode 
---------------------------------------------
0 | model | Sequential | 17.6 K | train
---------------------------------------------
17.6 K    Trainable params
0         Non-trainable params
17.6 K    Total params
0.070     Total estimated model params size (MB)
19        Modules in train mode
0         Modules in eval mode


iNaturalistModel(
  (model): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): Mish()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): Mish()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): Mish()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): Mish()
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): Mish()
    (14): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (15): Flatten(start_dim=1, end_dim=-1)
    (16): Dropout(p=0.2, inplace=Fals

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


epoch,▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▅▅▆▆▆▆▆▆▆▆▆▇▇▇▇▇████
train_acc,▃▂▃▂▂▃▃▆▃▆▆▄▆▃▆▁▃▇▆▃▆▅▆▅▅▇▃█▃▇█▆▅▃▅▅█▆▅▆
train_loss,██▆▇█▆█▇▆▆▆▅▅▄▅▇▇▃▃▇▂▄▁▄▃▂▇▁█▃▁▄▅▅▆▄▂▃▃▄
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇██
validation_acc,▁▃▆▇▆█▆███
validation_loss,█▆▄▃▃▂▃▁▁▁
epoch,9
train_acc,0.35484
train_loss,1.98512
trainer/global_step,2499
validation_acc,0.301


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.



## Question 3 (15 Marks)
Based on the above plots write down some insightful observations. For example,
- adding more filters in the initial layers is better
- Using bigger filters in initial layers and smaller filters in latter layers is better
- ... ...

(Note: I don't know if any of the above statements is true. I just wrote some random comments that came to my mind)



-
-
-
-
-



## Question 4 (5 Marks)
You will now apply your best model on the test data (You shouldn't have used test data so far. All the above experiments should have been done using train and validation data only).

- Use the best model from your sweep and report the accuracy on the test set.
- Provide a $10 \times 3$ grid containing sample images from the test data and predictions made by your best model (more marks for presenting this grid creatively).
- **(UNGRADED, OPTIONAL)** Visualise all the filters in the first layer of your best model for a random image from the test set. If there are 64 filters in the first layer plot them in an $8 \times 8$ grid.
- **(UNGRADED, OPTIONAL)** Apply guided back-propagation on any $10$ neurons in the CONV5 layer and plot the images which excite this neuron. The idea again is to discover interesting patterns which excite some neurons. You will draw a $10 \times 1$ grid below with one image for each of the $10$ neurons.




## Question 5 (10 Marks)
Paste a link to your github code for Part A

Example: https://github.com/&lt;user-id&gt;/da6401_assignment2/partA;

- We will check for coding style, clarity in using functions and a ```README``` file with clear instructions on training and evaluating the model (the 10 marks will be based on this).
- We will also run a plagiarism check to ensure that the code is not copied (0 marks in the assignment if we find that the code is plagiarised).
- We will also check if the training and test data has been split properly and randomly. You will get 0 marks on the assignment if we find any cheating (e.g., adding test data to training data) to get higher accuracy.



# Part B : Fine-tuning a pre-trained model

## Question 1 (5 Marks)
In most DL applications, instead of training a model from scratch, you would use a model pre-trained on a similar/related task/dataset. From ```torchvision```, you can load **ANY ONE** [model](https://pytorch.org/vision/stable/models.html) (```GoogLeNet```, ```InceptionV3```, ```ResNet50```, ```VGG```, ```EfficientNetV2```, ```VisionTransformer``` etc.) pre-trained  on the ImageNet dataset. Given that ImageNet also contains many animal images, it stands to reason that using a model pre-trained on ImageNet maybe helpful for this task.

You will load a pre-trained model and then fine-tune it using the naturalist data that you used in the previous question. Simply put, instead of randomly initialising the weigths of a network you will use the weights resulting from training the model on the ImageNet data (```torchvision``` directly provides these weights). Please answer the following questions:

- The dimensions of the images in your data may not be the same as that in the ImageNet data. How will you address this?
- ImageNet has $1000$ classes and hence the last layer of the pre-trained model would have $1000$ nodes. However, the naturalist dataset has only $10$ classes. How will you address this?

(Note: This question is only to check the implementation. The subsequent questions will talk about how exactly you will do the fine-tuning.)



Answer 1:\
One option is to resize the input image to fit into the size of the input by using transforms like resize or crop(which may cause the data to be lost)\
Also many models these days make use of adaptive pooling layer so that before flatterinig the image size does not affect the shapes and just before flattening the output get converted to a fixed size
**Answer 2**:\
This can be done by different ways
one way is to use another layer which convert this 1000 class into 10 class for which we need to learn the weights\
But a commongly employed way in case of transfer learning of these kind of problems is to keep only the initial CNN and Pooling layer and change the final fully connected layer to suit the need of the application that is change the final layer so that the output is only 10 classes and learn these weights with finetuning


In [ ]:
weights=torchvision.models.ViT_B_16_Weights.DEFAULT
auto_transforms=weights.transforms()
model=torchvision.models.vit_b_16(weights=weights)

Downloading: "https://download.pytorch.org/models/vit_b_16-c867db91.pth" to /root/.cache/torch/hub/checkpoints/vit_b_16-c867db91.pth
100%|██████████| 330M/330M [00:05<00:00, 66.2MB/s]


In [ ]:
auto_transforms

ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)

In [ ]:
import torch.nn as nn
model.heads=nn.Sequential(nn.Linear(in_features=768, out_features=10))
model

VisionTransformer(
  (conv_proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
  (encoder): Encoder(
    (dropout): Dropout(p=0.0, inplace=False)
    (layers): Sequential(
      (encoder_layer_0): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_attention): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
        )
        (dropout): Dropout(p=0.0, inplace=False)
        (ln_2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): MLPBlock(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU(approximate='none')
          (2): Dropout(p=0.0, inplace=False)
          (3): Linear(in_features=3072, out_features=768, bias=True)
          (4): Dropout(p=0.0, inplace=False)
        )
      )
      (encoder_layer_1): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_a

In [ ]:
from torchvision import models
class VIT_iNaturalist(pl.LightningModule):
    def __init__(self,num_classes=10,optimizer=torch.optim.Adam,lr=0.001,weights=models.ViT_B_16_Weights.DEFAULT):
      super().__init__()
      self.optimizer=optimizer
      self.lr=lr
      self.model =models.vit_b_16(weights=weights)
      self.model.heads=nn.Sequential(nn.Linear(in_features=768, out_features=num_classes))
    def forward(self, x):
        return self.model(x)
    def training_step(self, batch, batch_idx):
        images, labels = batch
        logits = self.forward(images)
        loss = nn.functional.cross_entropy(logits, labels)
        preds = torch.argmax(logits, dim=1)
        acc = (preds == labels).float().mean()
        self.log("train_acc", acc, prog_bar=True)
        self.log("train_loss", loss, prog_bar=True)
        return loss
    def validation_step(self, batch, batch_idx):
        images, labels = batch
        logits = self.forward(images)
        loss = nn.functional.cross_entropy(logits, labels)
        preds = torch.argmax(logits, dim=1)
        acc = (preds == labels).float().mean()
        self.log("validation_acc", acc, prog_bar=True)
        self.log("validation_loss", loss, prog_bar=True)
        return {"validation_loss":loss,"validation_acc":acc}
    def test_step(self, batch, batch_idx):
        images, labels = batch
        logits = self.forward(images)
        loss = nn.functional.cross_entropy(logits, labels)
        preds = torch.argmax(logits, dim=1)
        acc = (preds == labels).float().mean()
        self.log("test_acc", acc, prog_bar=True,)
        self.log("test_loss", loss, prog_bar=True)
        return {"test_loss":loss,"test_acc":acc}
    def configure_optimizers(self):
        return self.optimizer(self.model.parameters(), lr=self.lr,weight_decay=1e-5)

In [ ]:

class Transfer_iNaturalistModel(pl.LightningModule):
    def __init__(self,num_classes=10,optimizer=torch.optim.Adam,lr=0.001,base=models.resnet50(weights="DEFAULT")):
      super().__init__()
      self.optimizer=optimizer
      self.lr=lr
      num_filters = base.fc.in_features
      layers = list(base.children())[:-1]
      self.feature_extractor = nn.Sequential(*layers)
      self.model = nn.Linear(num_filters, num_classes)
    def forward(self, x):
        x=self.feature_extractor(x)
        x=torch.flatten(x,start_dim=1)
        return self.model(x)
    def training_step(self, batch, batch_idx):
        images, labels = batch
        logits = self.forward(images)
        loss = nn.functional.cross_entropy(logits, labels)
        preds = torch.argmax(logits, dim=1)
        acc = (preds == labels).float().mean()
        self.log("train_acc", acc, prog_bar=True)
        self.log("train_loss", loss, prog_bar=True)
        return loss
    def validation_step(self, batch, batch_idx):
        images, labels = batch
        logits = self.forward(images)
        loss = nn.functional.cross_entropy(logits, labels)
        preds = torch.argmax(logits, dim=1)
        acc = (preds == labels).float().mean()
        self.log("validation_acc", acc, prog_bar=True)
        self.log("validation_loss", loss, prog_bar=True)
        return {"validation_loss":loss,"validation_acc":acc}
    def test_step(self, batch, batch_idx):
        images, labels = batch
        logits = self.forward(images)
        loss = nn.functional.cross_entropy(logits, labels)
        preds = torch.argmax(logits, dim=1)
        acc = (preds == labels).float().mean()
        self.log("test_acc", acc, prog_bar=True)
        self.log("test_loss", loss, prog_bar=True)
        return {"test_loss":loss,"test_acc":acc}
    def configure_optimizers(self):
        return self.optimizer(self.model.parameters(), lr=self.lr,weight_decay=1e-5)

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 131MB/s]



## Question 2 (5 Marks)
You will notice that ```GoogLeNet```, ```InceptionV3```, ```ResNet50```, ```VGG```, ```EfficientNetV2```, ```VisionTransformer``` are very huge models as compared to the simple model that you implemented in Part A. Even fine-tuning on a small training data may be very expensive. What is a common trick used to keep the training tractable (you will have to read up a bit on this)? Try different variants of this trick and fine-tune the model using the iNaturalist dataset. For example, '_______'ing all layers except the last layer, '_______'ing upto $k$ layers and  '_______'ing the rest. Read up on pre-training and fine-tuning to understand what exactly these terms mean.

Write down the at least $3$ different strategies that you tried (simple bullet points would be fine).

- Freezing the pretrained feature extraction layer and training only the final classifier layer
- Freeze first k layer of the pretrained model and train remaining layers as well as the final fully connected layer
- Train the whole network without freezing anything


#### Training whole model

In [10]:
from torchvision import models
class VIT_iNaturalist_whole(pl.LightningModule):
    def __init__(self,num_classes=10,optimizer=torch.optim.Adam,lr=0.001,weights=models.ViT_B_16_Weights.DEFAULT):
      super().__init__()
      self.save_hyperparameters()
      self.optimizer=optimizer
      self.lr=lr
      self.model =models.vit_b_16(weights=weights)
      self.model.heads=nn.Sequential(nn.Linear(in_features=768, out_features=num_classes))
    def forward(self, x):
        return self.model(x)
    def training_step(self, batch, batch_idx):
        images, labels = batch
        logits = self.forward(images)
        loss = nn.functional.cross_entropy(logits, labels)
        preds = torch.argmax(logits, dim=1)
        acc = (preds == labels).float().mean()
        self.log("train_acc", acc, prog_bar=True)
        self.log("train_loss", loss, prog_bar=True)
        return loss
    def validation_step(self, batch, batch_idx):
        images, labels = batch
        logits = self.forward(images)
        loss = nn.functional.cross_entropy(logits, labels)
        preds = torch.argmax(logits, dim=1)
        acc = (preds == labels).float().mean()
        self.log("validation_acc", acc, prog_bar=True)
        self.log("validation_loss", loss, prog_bar=True)
        return {"validation_loss":loss,"validation_acc":acc}
    def test_step(self, batch, batch_idx):
        images, labels = batch
        logits = self.forward(images)
        loss = nn.functional.cross_entropy(logits, labels)
        preds = torch.argmax(logits, dim=1)
        acc = (preds == labels).float().mean()
        self.log("test_acc", acc, prog_bar=True,)
        self.log("test_loss", loss, prog_bar=True)
        return {"test_loss":loss,"test_acc":acc}
    def configure_optimizers(self):
        return self.optimizer(self.model.parameters(), lr=self.lr,weight_decay=1e-4)

In [ ]:
import wandb
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
import torchvision.models as models

wandb_logger = WandbLogger(project="DL_assignment2_B")
wandb.init(project="DL_assignment2_B", reinit=True)
weights = models.ViT_B_16_Weights.DEFAULT
auto_transforms = weights.transforms()
batch_size = 32

model=VIT_iNaturalist_whole(optimizer=torch.optim.Adam,lr=3e-4,weights=weights)
naturalist_DM=iNaturalistDataModule(train_dir='inaturalist_12K/train',test_dir='inaturalist_12K/val',batch_size=batch_size,train_transforms=auto_transforms,test_transforms=auto_transforms)

wandb_logger.experiment.config.update({
    "model_class": model.__class__.__name__
})
trainer = pl.Trainer(max_epochs=10, logger=wandb_logger)

trainer.fit(model, naturalist_DM)

wandb.watch(model, log="all", log_freq=100)

wandb.finish()


/usr/local/lib/python3.11/dist-packages/pytorch_lightning/utilities/parsing.py:209: Attribute 'train_transforms' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['train_transforms'])`.
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/utilities/parsing.py:209: Attribute 'test_transforms' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['test_transforms'])`.
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/loggers/wandb.py:397: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
INFO:pytorch_lightning.utilities.rank_zero:You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model r

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

#### Freezing first k layers

In [ ]:
from torchvision import models
class VIT_iNaturalist_first_k(pl.LightningModule):
    def __init__(self,num_classes=10, k:int=5,optimizer=torch.optim.Adam,lr=0.001,weights=models.ViT_B_16_Weights.DEFAULT):
      super().__init__()

      self.save_hyperparameters()
      self.optimizer=optimizer
      self.lr=lr
      self.model =models.vit_b_16(weights=weights)
      for param in self.model.parameters():
        param.requires_grad = False
      for param in self.model.encoder.layers[k:].parameters():
        param.requires_grad = True
      self.model.heads=nn.Sequential(nn.Linear(in_features=768, out_features=num_classes))
    def forward(self, x):
        return self.model(x)
    def training_step(self, batch, batch_idx):
        images, labels = batch
        logits = self.forward(images)
        loss = nn.functional.cross_entropy(logits, labels)
        preds = torch.argmax(logits, dim=1)
        acc = (preds == labels).float().mean()
        self.log("train_acc", acc, prog_bar=True)
        self.log("train_loss", loss, prog_bar=True)
        return loss
    def validation_step(self, batch, batch_idx):
        images, labels = batch
        logits = self.forward(images)
        loss = nn.functional.cross_entropy(logits, labels)
        preds = torch.argmax(logits, dim=1)
        acc = (preds == labels).float().mean()
        self.log("validation_acc", acc, prog_bar=True)
        self.log("validation_loss", loss, prog_bar=True)
        return {"validation_loss":loss,"validation_acc":acc}
    def test_step(self, batch, batch_idx):
        images, labels = batch
        logits = self.forward(images)
        loss = nn.functional.cross_entropy(logits, labels)
        preds = torch.argmax(logits, dim=1)
        acc = (preds == labels).float().mean()
        self.log("test_acc", acc, prog_bar=True,)
        self.log("test_loss", loss, prog_bar=True)
        return {"test_loss":loss,"test_acc":acc}
    def configure_optimizers(self):
        return self.optimizer(self.model.parameters(), lr=self.lr,weight_decay=1e-4)

In [9]:
import wandb
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
import torchvision.models as models


wandb_logger = WandbLogger(project="DL_assignment2_B")
wandb.init(project="DL_assignment2_B", reinit=True)

weights = models.ViT_B_16_Weights.DEFAULT
auto_transforms = weights.transforms()
batch_size = 64

model=VIT_iNaturalist_first_k(optimizer=torch.optim.Adam,lr=0.001,weights=weights,k=7)
naturalist_DM=iNaturalistDataModule(train_dir='inaturalist_12K/train',test_dir='inaturalist_12K/val',batch_size=batch_size,train_transforms=auto_transforms,test_transforms=auto_transforms)

wandb_logger.experiment.config.update({
    "model_class": model.__class__.__name__
})
trainer = pl.Trainer(max_epochs=10, logger=wandb_logger)

trainer.fit(model, naturalist_DM)

wandb.watch(model, log="all", log_freq=100)

wandb.finish()


/usr/local/lib/python3.11/dist-packages/pytorch_lightning/utilities/parsing.py:209: Attribute 'train_transforms' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['train_transforms'])`.
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/utilities/parsing.py:209: Attribute 'test_transforms' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['test_transforms'])`.
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/loggers/wandb.py:397: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
INFO:pytorch_lightning.utilities.rank_zero:You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model r

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


epoch,▁▁▁▂▂▂▂▃▃▃▃▃▃▃▄▄▄▅▅▅▅▆▆▆▆▆▆▆▇▇▇████
train_acc,▁▆▅▆▅▇▇▇▇▆▆▆▇▇▇▇▇▇▇▇▇█▇█▆
train_loss,█▄▄▄▅▂▂▂▂▂▃▃▂▂▂▂▂▂▂▂▂▁▂▁▂
trainer/global_step,▁▁▁▂▂▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇███
validation_acc,█▅▃█▆▁▅▄▂▅
validation_loss,▁▂▃▄▅▇▆▅█▇
epoch,9
train_acc,0.92063
train_loss,0.2049
trainer/global_step,1249
validation_acc,0.7995


In [ ]:
import wandb
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
import torchvision.models as models


wandb_logger = WandbLogger(project="DL_assignment2_B")
wandb.init(project="DL_assignment2_B", reinit=True)

weights = models.ViT_B_16_Weights.DEFAULT
auto_transforms = weights.transforms()
batch_size = 128

model=VIT_iNaturalist_first_k(optimizer=torch.optim.Adam,lr=0.001,weights=weights,k=3)
naturalist_DM=iNaturalistDataModule(train_dir='inaturalist_12K/train',test_dir='inaturalist_12K/val',batch_size=batch_size,train_transforms=auto_transforms,test_transforms=auto_transforms)

wandb_logger.experiment.config.update({
    "model_class": model.__class__.__name__
})
trainer = pl.Trainer(max_epochs=10, logger=wandb_logger)

trainer.fit(model, naturalist_DM)

wandb.watch(model, log="all", log_freq=100)

wandb.finish()


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: cs24m041 (cs24m041-iit-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


epoch,▁▁▂▂▃▃▃▃▃▄▄▅▅▆▆▆▆▆▇▇██
train_acc,▁▂▃▅▄▆▇█▇▇▇█
train_loss,█▆▆▃▅▃▂▁▁▂▂▁
trainer/global_step,▁▁▂▂▂▃▃▃▃▄▄▅▅▅▆▆▆▆▇▇██
validation_acc,▁▅▆▇▇▇▆▇█▇
validation_loss,▄▂▁▁▃▅▇▅██
epoch,9
train_acc,0.94531
train_loss,0.1469
trainer/global_step,629
validation_acc,0.619


#### Train only the dense layer

In [ ]:
from torchvision import models
class VIT_iNaturalist_dense_only(pl.LightningModule):
    def __init__(self,num_classes=10,optimizer=torch.optim.Adam,lr=0.001,weights=models.ViT_B_16_Weights.DEFAULT):
      super().__init__()

      self.save_hyperparameters()
      self.optimizer=optimizer
      self.lr=lr
      self.model =models.vit_b_16(weights=weights)
      for param in self.model.parameters():
        param.requires_grad = False
      self.model.heads=nn.Sequential(nn.Linear(in_features=768, out_features=num_classes))
    def forward(self, x):
        return self.model(x)
    def training_step(self, batch, batch_idx):
        images, labels = batch
        logits = self.forward(images)
        loss = nn.functional.cross_entropy(logits, labels)
        preds = torch.argmax(logits, dim=1)
        acc = (preds == labels).float().mean()
        self.log("train_acc", acc, prog_bar=True)
        self.log("train_loss", loss, prog_bar=True)
        return loss
    def validation_step(self, batch, batch_idx):
        images, labels = batch
        logits = self.forward(images)
        loss = nn.functional.cross_entropy(logits, labels)
        preds = torch.argmax(logits, dim=1)
        acc = (preds == labels).float().mean()
        self.log("validation_acc", acc, prog_bar=True)
        self.log("validation_loss", loss, prog_bar=True)
        return {"validation_loss":loss,"validation_acc":acc}
    def test_step(self, batch, batch_idx):
        images, labels = batch
        logits = self.forward(images)
        loss = nn.functional.cross_entropy(logits, labels)
        preds = torch.argmax(logits, dim=1)
        acc = (preds == labels).float().mean()
        self.log("test_acc", acc, prog_bar=True,)
        self.log("test_loss", loss, prog_bar=True)
        return {"test_loss":loss,"test_acc":acc}
    def configure_optimizers(self):
        return self.optimizer(self.model.parameters(), lr=self.lr,weight_decay=1e-4)

In [ ]:
import wandb
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
import torchvision.models as models

wandb_logger = WandbLogger(project="DL_assignment2_B")
weights = models.ViT_B_16_Weights.DEFAULT
auto_transforms = weights.transforms()
batch_size = 128
wandb.init(project="DL_assignment2_B", reinit=True)
model=VIT_iNaturalist_dense_only(optimizer=torch.optim.Adam,lr=0.001,weights=weights)
naturalist_DM=iNaturalistDataModule(train_dir='inaturalist_12K/train',test_dir='inaturalist_12K/val',batch_size=batch_size,train_transforms=auto_transforms,test_transforms=auto_transforms)

wandb_logger.experiment.config.update({
    "model_class": model.__class__.__name__
})
trainer = pl.Trainer(max_epochs=10, logger=wandb_logger)

trainer.fit(model, naturalist_DM)

wandb.watch(model, log="all", log_freq=100)

wandb.finish()


/usr/local/lib/python3.11/dist-packages/pytorch_lightning/loggers/wandb.py:397: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
INFO:pytorch_lightning.utilities.rank_zero:You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type              | Params | Mode 
----------------------------------------------------
0 | model | VisionTransformer | 85.8 M | train
--------------------------------

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:
Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined



## Question 3 (10 Marks)
Now fine-tune the model using **ANY ONE** of the listed strategies that you discussed above. Based on these experiments write down some insightful inferences comparing training from scratch and fine-tuning a large pre-trained model.


- From the graphs it is visible that for almost same parameter just differing the number of layer trained a almost perfect ordering based on the number of layers trained
- As the number of layer that need to be trained decrease the batch size supported in a fixed memory increase also the amount of time required to run the experiment decrease
Training only the final layer
- More faster computations as the amount of computation need to be done is limited to the final layer
- Better performance due to the pretrained layers which are optimized for much larger dataset size
- In training from scratch, due to the number of parametes there are much more precks and vallys which may cause the model to get stuck in but in case of using pretrained model some exploration has been alsready done on these higher dimensional space and like better weight initialization technique it help in reducing the amount of time required to reach or more easily abstract the hidden relation from the  fine tuning data

Choosing to freeze all the encoder block and finetune only the final classifier head of VIT model


## Question 4 (10 Marks)
Paste a link to your GitHub code for Part B

Example: https://github.com/&lt;user-id&gt;/da6401_assignment2/partB

Follow the same instructions as in Question 5 of Part A.



# (UNGRADED, OPTIONAL) Part C : Using a pre-trained model as it is

## Question 1 (0 Marks)
Object detection is the task of identifying objects (such as cars, trees, people, animals) in images. Over the past 6 years, there has been tremendous progress in object detection with very fast and accurate models available today. In this question you will use a pre-trained YoloV3 model and use it in an application of your choice. Here is a cool demo of YoloV2 (click on the image to see the demo on youtube).

[![IMAGE ALT TEXT HERE](https://img.youtube.com/vi/VOC3huqHrss/0.jpg)](https://www.youtube.com/watch?v=VOC3huqHrss)


Go crazy and think of a cool application in which you can use object detection (alerting lab mates of monkeys loitering outside the lab, detecting cycles in the CRC corridor, ....).

Make a similar demo video of your application, upload it on youtube and paste a link below (similar to the demo I have pasted above).

Also note that I do not expect you to train any model here but just use an existing model as it is. However, if you want to fine-tune the model on some application-specific data then you are free to do that (it is entirely up to you).

Notice that for this question I am not asking you to provide a GitHub link to your code. I am giving you a free hand to take existing code and tweak it for your application. Feel free to paste the link of your code here nonetheless (if you want).

Example: https://github.com/&lt;user-id&gt;/da6401_assignment2/partC